In [12]:
import tensorflow as tf
import cv2
import numpy as np
import os

In [13]:
input_shape = (16, 128, 128, 3)
# Define the encoder
def create_unet(input_shape):
    input = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same')(input)
    x = tf.keras.layers.MaxPooling3D((2, 2, 2), padding='same')(x)
    x = tf.keras.layers.Conv3D(128, (3, 3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.UpSampling3D((2, 2, 2))(x)
    output = tf.keras.layers.Conv3D(3, (3, 3, 3), activation='sigmoid', padding='same')(x)
    
    model = tf.keras.Model(input, output)
    return model
model=create_unet(input_shape)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 16, 128, 128,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d (Conv3D)                 │ (None, 16, 128, 128,   │         5,248 │
│                                 │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (None, 8, 64, 64, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 8, 64, 64, 128) │       221,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_2 (Conv3D)               │ (None, 8, 64, 64, 64)  │       221,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling3d (UpSampling3D)    │ (None, 16, 128, 128,   │             0 │
│                                 │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_3 (Conv3D)               │ (None, 16, 128, 128,   │         5,187 │
│                                 │ 3)                     │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 452,995 (1.73 MB)

 Trainable params: 452,995 (1.73 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(optimizer='adam', loss='mse')

In [15]:
# Function to load and preprocess videos
def load_videos(folder, frame_size, frame_count):
    frames = []
    for filename in os.listdir(folder):
        if filename.endswith('.avi'):
            cap = cv2.VideoCapture(os.path.join(folder, filename))
            video_frames = []
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, frame_size)
                video_frames.append(frame)
                if len(video_frames) == frame_count:
                    frames.append(video_frames)
                    video_frames = []
            cap.release()
    frames = np.array(frames)
    frames = frames / 255.0  # Normalize
    return frames

# Load and preprocess videos
frame_size = (128, 128)
frame_count = 16
video_frames = load_videos('h1/brush_hair', frame_size, frame_count)

In [ ]:
# Train the autoencoder
model.fit(video_frames, video_frames, epochs=10, batch_size=1)


Epoch 1/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 151s 2s/step - loss: 0.0570
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 207s 2s/step - loss: 0.0096
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 256s 3s/step - loss: 0.0044
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 260s 3s/step - loss: 0.0046
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 334s 3s/step - loss: 0.0035
Epoch 6/10
20/97 ━━━━━━━━━━━━━━━━━━━━ 5:11 4s/step - loss: 0.0019

In [ ]:
model.save('mygg.h5')

In [ ]:
# Function to save video
def save_video(filename, frames, frame_size):
    height, width = frame_size
    video = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height))
    for frame_set in frames:
        for frame in frame_set:
            frame = (frame * 255).astype(np.uint8)
            video.write(frame)
    video.release()

# Save reconstructed video
save_video('reconstructed_cat2.mp4', video_frames, frame_size)

print("Video processing complete.")

Video processing complete.
